# Load data into local PostgreSQL database

Create a PostgreSQL database and populate a table using a csv file.

Make sure you have [postgres installed on your machine](https://blog.timescale.com/tutorials/how-to-install-psql-on-mac-ubuntu-debian-windows/), [your server is running](https://www.postgresql.org/docs/9.1/server-start.html) and [your path variable is set](https://www.postgresql.org/docs/9.1/install-post.html). 

Once you have done that and you find yourself at the psql command line (`postgres=+`), create a database. Somewhat paradoxically, you need to have a pre-existing database to connect with psycopg2 in order to create a database. Gotta have a db to make a db. You can make a database from the postgres command line like so:

```
CREATE DATABASE music_test;
```

If you want to give another user access to this database you can do so with the following commands:

```
CREATE USER music_man with encrypted password 'epic_shredder_brah';
GRANT ALL PRIVILEGES ON DATABASE music_test TO music_man;
```

To confirm that these commands worked, you can use the `\l` command from the postgres command line:

```
postgres=# \l
                                List of databases
    Name    |   Owner    | Encoding | Collate | Ctype |     Access privileges     
------------+------------+----------+---------+-------+---------------------------
 music_test | colinspear | UTF8     | C       | C     | =Tc/colinspear           +
            |            |          |         |       | colinspear=CTc/colinspear+
            |            |          |         |       | music_man=CTc/colinspear
 postgres   | colinspear | UTF8     | C       | C     | 
 
```

If you also want this user to be able to create databases, you will will also need to do that explicitly:

```
ALTER USER music_man CREATEDB;
```

If that command returns `ALTER ROLE`, you should be in business. You can also see what roles exist, their permissions and what groups they are a part of on your server with the `\dg` command:

```
postgres=# \dg
                                    List of roles
 Role name  |                         Attributes                         | Member of 
------------+------------------------------------------------------------+-----------
 colinspear | Superuser, Create role, Create DB, Replication, Bypass RLS | {}
 music_man  | Create DB                                                  | {}
 ```

If it's just you that will be using the database, you should just be able to use the master account you created when installing PostreSQL. If you are looking for a good introduction to PostgreSQL roles and users, I found [this article](https://aws.amazon.com/blogs/database/managing-postgresql-users-and-roles/) super informative.

Now that that's all done, let's make a databse and a table with psycopg2! First we will import the libraries we need and make our connection to the database we created above:

In [3]:
from psycopg2 import connect, extensions

con = connect(
    dbname='music_test',
    user='music_man', 
    host='localhost',
    password='epic_shredder_brah'
)

There are two main psycopg2 objects we'll be interested in here: the connection (which we created above) and the cursor (which we will create in just a minute). The connection establishes our phone line to the database (creates a *persistent client session*), while the cursor is used by the connection to place our call. In other words, the connection object establishes a connection, while the cursor executes the commands. In order to get our lackey the cursor to shlep our commands back and forth, we have the `execute()` method to which we feed our SQL queries. To illustrate all of this, we will make a simple users table we can insert some fake data into:

In [2]:
cursor = con.cursor()

In [3]:
cursor.execute("""
    create table users(
        id integer PRIMARY KEY,
        email text,
        name text,
        address text
)
""")

One way to add rows to the database is by passing a SQL command as a string to `execute()`. You can do this one row at a time, typing in the values, or you can package your data in a list of tuples and loop over this list using as shown below. Once we have inserted all of the data

In [4]:
cursor.execute(
    "INSERT INTO users VALUES (1, 'test@example.ca', 'Kamala Harris', '123 B St.')"
)

This works fine if you want to write your own SQL statements, but there will probably come a time where you have some data and it doesn't make sense to write it all in to a SQL command. For this there are other methods, such as `execute_many` which takes a list of tuples containing your data and inserts them into your database with one command:

In [5]:
data = [
    (11, "hello@world.atom", "Quark Fromage", "999 Entropy Blvd."),
    (12, "rodney@dangerfield.golf", "Rod Danger", "420 Beerthirty St."),
    (13, "another@email.fun", "Karen Park", "124 Another Lane")
]

cursor.executemany('INSERT INTO users VALUES (%s, %s, %s, %s)', data)

In [6]:
cursor.execute('select * from users')

The cursor will stockpile our queries until we tell it to go by calling the `commit()` method, at which point it will execute the queries in the order they were received. Use `rollback()` to discard the queries you have made since opening up the previous connection.

In [7]:
con.commit()

Hopefully, this will all have gone according to plan and you will be met with beautiful silence. But hold up. Where is our data? Turns out the cursor is still holding on to it. To fetch, we (fittingly) have the cursor methods `fetchone()` and `fetchall()`. Here I'll just grab it all. Careful though, you can only fetch once before your results are lost to the wind.

In [9]:
cursor.fetchall()

[]

In [10]:
con.close()

Now, what I really want to do is load the Spotify Discover Weekly data I have been collecting over the last year into a table. Instead of adding a table to the test database we've been using up to this point, we'll first create a new database, create a `songs` table and then use the `copy_from()` method to load a csv with the song data into a table. 

The first thing we have to do is change the isolation level setting. Basically isolation level concerns when and how changes are made to a database. It safeguards against multiple  conflicting changes being made to the database at once. The default for PostgreSQL servers is typically `READ COMMITTED`  which means any data read is committed at the moment it is read (i.e. when you call `commit()` on your connection). Whenever you open a new connection using psycopg2, you initiate a transaction. Whenever you call `commit()` the transaction is written.

In psycopg2, there are a few SQL commands that are not allowed to happen within a transaction - they need to be exectued as soon as they are passed to the `execute()` function. `CREATE DATABASE` is one of these commands. In order to make this happen, we need to set the isolation level to autocommit, which will do exactly that:

In [11]:
con = connect(
    dbname='music_test',
    user='music_man', 
    host='localhost',
    password='epic_shredder_brah'
)


con.set_isolation_level(extensions.ISOLATION_LEVEL_AUTOCOMMIT)
cur = con.cursor()

cur.execute("CREATE DATABASE discover_weekly")
con.close()

Closing the connection after creating the new database means I can open a new connection for making the songs table using the default isolation level.

Now we can connect to our new database, create and populate our table and check that it worked. I am going to use the psycopg2 objects as [context managers](https://docs.python.org/3/reference/datamodel.html#context-managers) which will close them immediately once we leave the with block. This way we don't have to worry about closing it manually.

In [42]:
conn = connect(
    database = 'discover_weekly',
    user = 'music_man',
    password = 'epic_shredder_brah',
    host = 'localhost'
)

with conn.cursor() as curs:
    curs.execute('''
        create table songs(
            time_added timestamp,
            release_date date,
            release_date_precision varchar,
            artist_name varchar,
            artist_id varchar,
            song_id varchar PRIMARY KEY,
            song_length_ms integer,
            song_name varchar,
            popularity integer,
            loudness real,
            tempo real,
            tempo_confidence real,
            time_signature integer,
            time_sig_conf real,
            key integer,
            key_confidence real,
            mode integer,
            mode_confidence real,
            danceability real,
            energy real,
            speechiness real,
            acousticness real,
            instrumentalness real,
            liveness real,
            valence real
        )
    ''')

    with open('../data/raw/dw_combined.csv') as f:
        next(f)
        curs.copy_from(f, 'songs', sep='\t')

conn.commit()

And to make sure that all worked, we can open a new cursor and fetch a row:

In [44]:
with conn.cursor() as curs:
    curs.execute('select * from songs limit 1')
    print(curs.fetchone())

(datetime.datetime(2019, 10, 21, 2, 30), datetime.date(2014, 6, 27), 'day', 'Woo', '5KqOWuKJPySBjp3mGVBWFR', '6K7LLBiLOcowTl3ECYxjRj', 217131, 'Make Me Tea', 35, -13.417, 151.858, 0.094, 4, 0.97, 10, 0.384, 1, 0.438, 0.553, 0.281, 0.0555, 0.976, 0.0205, 0.323, 0.501)


In [40]:
conn.commit()

In [28]:
curs.fetchone()

InterfaceError: cursor already closed

In [29]:
c.close()

## [Check this blog post out](https://www.alisa-in.tech/post/2021-01-18-de-project/) for an idea of how to show some stats about the loaded data.

In [14]:
import pandas as pd
df = pd.read_pickle('../data/raw/dw_combined.pkl')
df['time_added'] = pd.to_datetime(df['time_added'])
df['time_added'] = df['time_added'].dt.tz_localize(None)
df['release_date'] = pd.to_datetime(df['release_date'])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 29
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   time_added              300 non-null    datetime64[ns]
 1   release_date            300 non-null    datetime64[ns]
 2   release_date_precision  300 non-null    object        
 3   artist_name             300 non-null    object        
 4   artist_id               300 non-null    object        
 5   song_id                 300 non-null    object        
 6   song_length_ms          300 non-null    int64         
 7   song_name               300 non-null    object        
 8   popularity              300 non-null    int64         
 9   loudness                300 non-null    float64       
 10  tempo                   300 non-null    float64       
 11  tempo_confidence        300 non-null    float64       
 12  time_signature          300 non-null    int64      

In [32]:
df.to_csv('../data/raw/dw_combined.csv', sep='\t', index=False)

In [30]:
df[35:45]

,time_added,release_date,release_date_precision,artist_name,artist_id,song_id,song_length_ms,song_name,popularity,loudness,...,key_confidence,mode,mode_confidence,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence
5,2019-10-28 02:30:00,2015-03-23,day,Happyness,4nddgNqdf0SFeeqhbDN6rN,0P9ryrwnRQSTPsDXAonq8W,537159,Weird Little Birthday Girl,41,-14.731,...,0.550,1,0.671,0.641,0.307,0.0455,0.54000,0.864000,0.0915,0.294
6,2019-10-28 02:30:00,2012-10-01,day,Tim Maia,0jOs0wnXCu1bGGP7kh5uIu,41MeQZ4xHRLtPZQ50L2msF,240040,Where is My Other Half,32,-8.916,...,0.660,0,0.540,0.463,0.519,0.0418,0.79800,0.032800,0.1030,0.325
7,2019-10-28 02:30:00,2018-11-30,day,Foxwarren,2dPIBvg7mU59dCTGjhPylV,0W1vFF5qyLJUvQbTGOXVOW,296920,Fall Into A Dream,27,-9.976,...,0.626,0,0.622,0.748,0.356,0.0420,0.67300,0.087500,0.1760,0.149
8,2019-10-28 02:30:00,2014-08-04,day,Javier Bergia,00UEHtbE9afbslwaeYm9mw,1jrZQZNAQ5trykH0IGxTMR,319130,Midnight Round Mekines - Original Mix,34,-8.539,...,0.832,1,0.692,0.683,0.699,0.0353,0.34100,0.079000,0.1330,0.829
9,2019-10-28 02:30:00,2019-09-24,day,The Good Ones,7rIF4gQrWPRDL020SG2eKD,2bOklmZZSC5FiqVB13BXFZ,277800,"Where Did You Go Wrong, My Love",32,-8.447,...,0.821,0,0.728,0.519,0.495,0.0347,0.81600,0.000013,0.0977,0.634
10,2019-10-28 02:30:00,2018-07-18,day,Mildlife,1lsJmnN90u3KBvF3tGEWr3,4EsXLIwB7t00KH4iz8BVNA,682746,Phase II,7,-8.126,...,0.761,0,0.690,0.443,0.614,0.0375,0.27800,0.740000,0.0803,0.132
11,2019-10-28 02:30:00,2018-03-23,day,Bonny Doon,0AL8XXRh7fUrHAMLoDv9rn,0aEwbmIKDdv94O1sm38Y2B,198380,A Lotta Things,49,-7.979,...,0.667,1,0.686,0.482,0.472,0.0258,0.05840,0.169000,0.1120,0.341
12,2019-10-28 02:30:00,2019-07-10,day,Sam Evian,5MXsi1oKkm8LuStuKkMdzu,4UbBr8ZQB26757ah0zeRDU,267962,Right Down the Line,43,-8.637,...,0.473,1,0.571,0.750,0.649,0.0302,0.04830,0.040300,0.0784,0.606
13,2019-10-28 02:30:00,2016-09-30,day,Adam Torres,4wIdxySSxqlIirsqE0JKx8,112nYSRSK3kZjtJrzWRbdj,360946,Juniper Arms,8,-11.985,...,1.000,1,0.599,0.396,0.208,0.0282,0.86500,0.318000,0.1070,0.117
14,2019-10-28 02:30:00,2017-06-09,day,Supergombo,2IRev1maVOLaKbFofZ7AtB,4Wlwt72CajQUVcOt7IS9RW,322320,Marquis Warren,33,-6.338,...,0.633,1,0.631,0.611,0.829,0.0418,0.00368,0.672000,0.2900,0.823


[1] This is not necessarily trivial. Two things that may get you caught up areDon't forget to set your path variable (`export PATH=Path/to/PostgreSQL/Version/bin/:$PATH` at the command line). I also had some permissions errors and got around them by using sudo privileges: `sudo -u postgres psql`

[3] Thanks to [this article](https://kb.objectrocket.com/postgresql/create-a-postgresql-database-using-the-psycopg2-python-library-755) for these setup tips.